In [1]:
# !pip install transformers

In [1]:
import model_loader
import pipeline
from PIL import Image
from pathlib import Path
from transformers import CLIPTokenizer
import torch
import os

DEVICE = "cpu"

ALLOW_CUDA = True
ALLOW_MPS = False

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

tokenizer = CLIPTokenizer(os.getcwd() + "/data/tokenizer_vocab.json", merges_file=os.getcwd() + "/data/tokenizer_merges.txt")
model_file = os.getcwd() + "/data/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)

## TEXT TO IMAGE

# prompt = "A dog with sunglasses, wearing comfy hat, looking at camera, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
# prompt = "A cat stretching on the floor, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
prompt = "A modern glass skyscraper of 50 floors, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 8  # min: 1, max: 14

## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "../images/dog.jpg"
# input_image = Image.open(image_path)
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.9

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 42

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
)

# Combine the input image and the output image into a single image.
Image.fromarray(output_image)

c:\Users\Yash\Desktop\Projects\NN-from-scratch\nn-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


RuntimeError: Error(s) in loading state_dict for VAE_Encoder:
	Missing key(s) in state_dict: "1.conv1.weight", "1.conv1.bias", "1.conv2.weight", "1.conv2.bias", "2.conv1.weight", "2.conv1.bias", "2.conv2.weight", "2.conv2.bias", "4.conv1.weight", "4.conv1.bias", "4.conv2.weight", "4.conv2.bias", "5.conv1.weight", "5.conv1.bias", "5.conv2.weight", "5.conv2.bias", "7.conv1.weight", "7.conv1.bias", "7.conv2.weight", "7.conv2.bias", "8.conv1.weight", "8.conv1.bias", "8.conv2.weight", "8.conv2.bias", "10.conv1.weight", "10.conv1.bias", "10.conv2.weight", "10.conv2.bias", "11.conv1.weight", "11.conv1.bias", "11.conv2.weight", "11.conv2.bias", "12.conv1.weight", "12.conv1.bias", "12.conv2.weight", "12.conv2.bias", "14.conv1.weight", "14.conv1.bias", "14.conv2.weight", "14.conv2.bias". 
	Unexpected key(s) in state_dict: "1.conv_1.weight", "1.conv_1.bias", "1.conv_2.weight", "1.conv_2.bias", "2.conv_1.weight", "2.conv_1.bias", "2.conv_2.weight", "2.conv_2.bias", "4.conv_1.weight", "4.conv_1.bia